
#  Mini-Lab: MCP + LLM + NLP (com spaCy)

Neste exercício você vai aprender a:
1. Usar um modelo do Hugging Face para respostas livres.
2. Criar um mini-classificador de intenções com spaCy.
3. Simular um MCP (Model Context Protocol) para que o chatbot saiba quando chamar funções externas.

 Vamos lá!


In [ ]:

!pip install spacy transformers accelerate
!python -m spacy download pt_core_news_md


In [ ]:

import spacy
from transformers import pipeline


In [ ]:

# TODO: carregue o modelo Hugging Face em português
#generator =

# Carregar modelo spaCy em PT-BR
#nlp = 


In [ ]:

def get_weather(city):
    return f"A previsão para {city} é de 24°C e ensolarado."

def calculate(expression):
    try:
        result = eval(expression)  #função perigosa...
        return f"O resultado é {result}"
    except:
        return "Expressão inválida."


In [ ]:

# TODO: adicione pelo menos 3 exemplos para cada intenção
intents = {
    #"weather": ["previsão do tempo", ...],
    #"math": ["calcule", ...],
}


In [ ]:

#Criar embeddings de exemplo
intent_docs = {intent: [nlp(ex) for ex in examples] for intent, examples in intents.items()}

def classify_intent(text, threshold=0.70): #Alterar o threshold se necessário
    doc = nlp(text.lower())
    best_intent = None
    best_score = 0.0
    
    for intent, examples in intent_docs.items():
        for ex_doc in examples:
            score = doc.similarity(ex_doc)
            #print("item: " + str(ex_doc) + "      score: " + str(score)) #Descomente para debug
            if score > best_score:
                best_intent = intent
                best_score = score
    
    if best_score >= threshold:
        return best_intent
    return None

#Teste rápido (tentar obter um resultado coerente aqui) 
print(classify_intent("Qual a predição do clima amanhã?"))
print(classify_intent("Resolva 10 + 5"))


In [ ]:

def chatbot_with_mcp(user_input):
    intent = classify_intent(user_input)
    
    if intent == "weather":
        if "para" in user_input:
            city = user_input.split("para")[-1].strip().replace("?", "")
        else:
            city = "sua cidade"
        return f"(Ferramenta via MCP) {get_weather(city)}"
    
    elif intent == "math":
        expr = user_input.lower().replace("calcule", "").replace("faça a conta", "").strip()
        return f"(Ferramenta via MCP) {calculate(expr)}"
    
    else:
        response = generator(user_input, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
        return f"(LLM) {response}"


In [ ]:

print(chatbot_with_mcp("Qual a predição do clima para Porto Alegre?"))
print(chatbot_with_mcp("Me dê a antevisão do tempo amanhã"))
print(chatbot_with_mcp("Calcule 12 + 7"))
print(chatbot_with_mcp("calcule 15 * 3"))
print(chatbot_with_mcp("Me conte uma piada"))